In [1]:
# py35

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os

In [3]:
import itertools
import time
import random
import glob
import PIL
import asyncio
import multiprocessing as mp
import queue

In [34]:
# Cyclic simulation of image grabbing from a given mask (with waiting)

IM_MASK= r'/Users/alex/Dropbox/PhD/CODE/DATA/IMG/feeder2/*'

def event_generator(wait_func=lambda:time.sleep(random.random())):
    while True:
        wait_func()
        yield
        
def grab_image(events):
    images = itertools.cycle(glob.glob(IM_MASK))
    for ev in events:
        im_name = next(images)
        im = PIL.Image.open(im_name)
        yield im

        
images = grab_image(event_generator())

for i in range(5):
    im = next(images)
    print(im)

<PIL.BmpImagePlugin.BmpImageFile image mode=RGB size=1024x768 at 0x10A8D7A90>
<PIL.BmpImagePlugin.BmpImageFile image mode=RGB size=1024x768 at 0x10A7E4748>
<PIL.BmpImagePlugin.BmpImageFile image mode=RGB size=1024x768 at 0x10A8D7A90>
<PIL.BmpImagePlugin.BmpImageFile image mode=RGB size=1024x768 at 0x10A7E4748>
<PIL.BmpImagePlugin.BmpImageFile image mode=RGB size=1024x768 at 0x10A8D7A90>


In [ ]:
def coroutine(func):
    def wrapper(*args, **kvargs):
        cr = func(*args, **kvargs)
        next(cr)
        return cr
    return wrapper

In [36]:
# Push-based coroutines

@coroutine
def source_coroutine(target):
    while True:
        msg = (yield)
        target.send(msg.lower())   
        
@coroutine
def sink_coroutine():
    while True:
        msg = (yield)
        print(msg.split())
        
src = source_coroutine(sink_coroutine())
src.send('Hello world dfdf yall!')

['hello', 'world', 'dfdf', 'yall!']


In [32]:
# Pull-based generators 

def src_gen(initial_strings):
    # itertools.cycle(initial_strings)
    for s in initial_strings:
        time.sleep(0.5)
        print(time.time(), 'Yielding a string ', s)
        yield s
        
def lower_gen(in_g):
    for msg in in_g:
        time.sleep(1.0)
        print(time.time(), 'Transforming ', msg, ' to lowercase')
        yield msg.lower()
        
def splitter_gen(in_g):
    for msg in in_g:
        print(time.time(), 'Splitting ', msg)
        yield msg.split()


In [37]:
def test_generators():
    initial_strings = ['Hello world!', 'Its beginning to look like Christmas...', 'Bello! Banana!']

    for res in splitter_gen(lower_gen(src_gen(initial_strings))):
        print(time.time(), 'Received ', res) 
        
test_generators()

1474569038.272593 Yielding a string  Hello world!
1474569039.274952 Transforming  Hello world!  to lowercase
1474569039.275097 Splitting  hello world!
1474569039.275122 Received  ['hello', 'world!']
1474569039.779384 Yielding a string  Its beginning to look like Christmas...
1474569040.783468 Transforming  Its beginning to look like Christmas...  to lowercase
1474569040.783701 Splitting  its beginning to look like christmas...
1474569040.783732 Received  ['its', 'beginning', 'to', 'look', 'like', 'christmas...']
1474569041.2873 Yielding a string  Bello! Banana!
1474569042.292056 Transforming  Bello! Banana!  to lowercase
1474569042.292252 Splitting  bello! banana!
1474569042.292307 Received  ['bello!', 'banana!']
